<a href="https://colab.research.google.com/github/nazalnihad/AI_for_good/blob/main/table_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install camelot-py[cv] tabula-py pdfplumber


In [2]:
!apt-get install ghostscript python3-tk


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.6).
python3-tk is already the newest version (3.10.8-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [3]:
import camelot
import tabula
import pdfplumber

# Function to extract tables using Camelot
def extract_tables_camelot(pdf_path):
    tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
    return tables

# Function to extract tables using Tabula
def extract_tables_tabula(pdf_path):
    tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, output_format='dataframe')
    return tables

# Function to extract tables using PDFPlumber
def extract_tables_pdfplumber(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_tables = page.extract_tables()
            for table in page_tables:
                tables.append(table)
    return tables


In [ ]:
!pip install PyPDF2==1.26.0


In [5]:
pdf_path = '/content/drive/MyDrive/AI_for_good/MSc5yrCSSchemeandSyllabus2021.pdf'

# Extract tables using different methods
tables_camelot = extract_tables_camelot(pdf_path)
tables_tabula = extract_tables_tabula(pdf_path)
tables_pdfplumber = extract_tables_pdfplumber(pdf_path)

# You can now process these tables further as per your requirements


ZeroDivisionError: ignored

In [6]:
import pandas as pd

In [7]:
# File path for the new PDF
file_path_new = '/content/drive/MyDrive/AI_for_good/nutrition_tb.pdf'

# Extracting tables from the new PDF using PDFPlumber
with pdfplumber.open(file_path_new) as pdf:
    extracted_tables_pdfplumber_new = []
    for page in pdf.pages:
        # Extract tables from each page
        page_tables = page.extract_tables()
        for table in page_tables:
            # Convert each extracted table to a DataFrame
            df = pd.DataFrame(table[1:], columns=table[0])
            extracted_tables_pdfplumber_new.append(df)

# Display the first few tables as an example
#  Displaying first 3 tables as examples



TypeError: ignored

In [12]:
for table in extracted_tables_pdfplumber_new:
  print(table)  #

KeyError: ignored

In [ ]:
import openai
import pdfplumber
import re  # For regex to find page numbers in the query

# Function to extract text from a PDF with page numbers
def extract_text_by_page(pdf_path):
    page_texts = {}
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            page_texts[i + 1] = page.extract_text()  # Storing text with page number as key
    return page_texts

# Function to find page number in user query
def find_page_number_in_query(query):
    match = re.search(r'page (\d+)', query, re.I)
    if match:
        return int(match.group(1))
    return None

# Function to query OpenAI with a specific chunk of text
def query_openai(text, user_query):
    openai.api_key = 'your-api-key'
    prompt = f"Based on the following text from the requested page:\n{text}\n\nQ: {user_query}\nA:"
    response = openai.Completion.create(
        engine="davinci",
        prompt=prompt,
        max_tokens=150,
        temperature=0.3,
        stop=["\n", " Q:", " A:"]
    )
    return response.choices[0].text.strip()

# Main function to find relevant text based on page number and get response
def answer_question_from_pdf(pdf_path, user_query):
    page_texts = extract_text_by_page(pdf_path)
    requested_page_number = find_page_number_in_query(user_query)

    if requested_page_number and requested_page_number in page_texts:
        page_text = page_texts[requested_page_number]
        answer = query_openai(page_text, user_query)
        return answer, pdf_path, requested_page_number
    else:
        return "Requested page number not found or not specified in the query.", pdf_path, None

# Example usage
pdf_path = 'path_to_your_pdf.pdf'
user_query = 'What is the main topic discussed on page 10?'
answer, filename, page_number = answer_question_from_pdf(pdf_path, user_query)
print(f"Answer: {answer}\nFile: {filename}\nPage: {page_number if page_number else 'Not specified'}")
